### SQL Assignment - sqlalchemy

#### Problem Statement:

Read the following data set:
https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data

Task:
1. Create an sqlalchemy engine using a sample from the data set
2. Write two basic update queries
3. Write two delete queries
4. Write two filter queries
5. Write two function queries

In [1]:
import pandas as pd
from numpy import genfromtxt
from time import time
from datetime import datetime
from sqlalchemy import Column, Integer, Float, Date, String, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [2]:
url1 = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
url2 = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names'
cols = ["age", "type_employer", "fnlwgt", "education", 
                "education_num","marital", "occupation", "relationship", "race","sex",
                "capital_gain", "capital_loss", "hr_per_week","country", "income"]
df = pd.read_csv(url1,sep = ',',names = cols)
df.head()

,age,type_employer,fnlwgt,education,education_num,marital,occupation,relationship,race,sex,capital_gain,capital_loss,hr_per_week,country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age              32561 non-null int64
type_employer    32561 non-null object
fnlwgt           32561 non-null int64
education        32561 non-null object
education_num    32561 non-null int64
marital          32561 non-null object
occupation       32561 non-null object
relationship     32561 non-null object
race             32561 non-null object
sex              32561 non-null object
capital_gain     32561 non-null int64
capital_loss     32561 non-null int64
hr_per_week      32561 non-null int64
country          32561 non-null object
income           32561 non-null object
dtypes: int64(6), object(9)
memory usage: 2.6+ MB


In [11]:
df.to_csv("adult.csv")

In [3]:
def Load_Data(file_name):
    data = genfromtxt(file_name, delimiter=',', skip_header=1, converters={0: lambda s: str(s)})
    return data.tolist()

Base = declarative_base()

class adult_data(Base):
    __tablename__ = 'adult'
    __table_args__ = {'sqlite_autoincrement': True}
    #tell SQLAlchemy the name of column and its attributes:
    id = Column(Integer, primary_key=True, nullable=False) 
    age = Column(Integer)
    type_employer = Column(String)
    fnlwgt = Column(Integer)
    education = Column(String)
    sex = Column(String)
    income = Column(String)

if __name__ == "__main__":
    t = time()

    #Create the database
    engine = create_engine('sqlite:///csv_test.db')
    Base.metadata.create_all(engine)

    #Create the session
    session = sessionmaker()
    session.configure(bind=engine)
    s = session()

    try:
        file_name = "adult.csv" #sample CSV file used:  http://www.google.com/finance/historical?q=NYSE%3AT&ei=W4ikVam8LYWjmAGjhoHACw&output=csv
        data = Load_Data(file_name) 

        for i in data:
            record = adult(**{
                'id': i[1],
                'age' : i[2],
                'type_employer' : i[3],
                'fnlwgt' : i[4],
                'education' : i[5],
                'sex' : i[6],
                'income':i[7]
            })
            s.add(record) #Add all the records

        s.commit() #Attempt to commit all the records
    except:
        s.rollback() #Rollback the changes on error
    finally:
        s.close() #Close the connection
    print ("Time elapsed: " + str(time() - t) + " s.") #0.091s

Time elapsed: 1.6436333656311035 s.


In [4]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///csv_test.db', echo=True)

In [5]:
Base = declarative_base()

In [6]:
class adult_test(Base):
...     __tablename__ = 'adult'
...     __table_args__ = {'extend_existing': True}
...     id = Column(Integer, primary_key=True)
...     age = Column(Integer)
...     type_employer = Column(String)
...     fnlwgt = Column(String(20))
...     education = Column(String(20))
...     sex = Column(String(20))
...     income = Column(String(20))
...
...     def __repr__(self):
...        return "<adult_data(age='%d',fnlwgt='%d',education='%s',sex='%s',income='%s')>" % (
...                             self.age, self.fnlwgt, self.education, self.sex, self.income)

In [7]:
adult_data.__table__

Table('adult', MetaData(bind=None), Column('id', Integer(), table=<adult>, primary_key=True, nullable=False), Column('age', Integer(), table=<adult>), Column('type_employer', String(), table=<adult>), Column('fnlwgt', Integer(), table=<adult>), Column('education', String(), table=<adult>), Column('sex', String(), table=<adult>), Column('income', String(), table=<adult>), schema=None)

In [8]:
adult_test.__table__

Table('adult', MetaData(bind=None), Column('id', Integer(), table=<adult>, primary_key=True, nullable=False), Column('age', Integer(), table=<adult>), Column('type_employer', String(), table=<adult>), Column('fnlwgt', String(length=20), table=<adult>), Column('education', String(length=20), table=<adult>), Column('sex', String(length=20), table=<adult>), Column('income', String(length=20), table=<adult>), schema=None)

In [9]:
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [10]:
Base.metadata.create_all(engine)

2018-06-13 23:44:46,574 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-06-13 23:44:46,581 INFO sqlalchemy.engine.base.Engine ()
2018-06-13 23:44:46,591 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-06-13 23:44:46,596 INFO sqlalchemy.engine.base.Engine ()
2018-06-13 23:44:46,602 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("adult")
2018-06-13 23:44:46,609 INFO sqlalchemy.engine.base.Engine ()


In [18]:
ed_adult = adult_test(age=23, fnlwgt=71519, type_employer=' Private', education=' Bachelors',sex=' Male' , income=' >=50k')

In [19]:
print(ed_adult)

<adult_data(age='23',fnlwgt='71519',education=' Bachelors',sex=' Male',income=' >=50k')>


In [20]:
print("adult age {}\n adult fnlwgt {}\n adult type_employer {}\n adult education {}\n adult sex {}\n adult income {}". 
      format(ed_adult.age, ed_adult.fnlwgt, ed_adult.type_employer, ed_adult.education,ed_adult.sex, ed_adult.income))

adult age 23
 adult fnlwgt 71519
 adult type_employer  Private
 adult education  Bachelors
 adult sex  Male
 adult income  >=50k


#### Add data

In [21]:
session.add(ed_adult)

In [22]:
our_cand = session.query(adult_data).all()

2018-06-13 23:46:11,333 INFO sqlalchemy.engine.base.Engine INSERT INTO adult (age, type_employer, fnlwgt, education, sex, income) VALUES (?, ?, ?, ?, ?, ?)
2018-06-13 23:46:11,343 INFO sqlalchemy.engine.base.Engine (23, ' Private', 71519, ' Bachelors', ' Male', ' >=50k')
2018-06-13 23:46:11,348 INFO sqlalchemy.engine.base.Engine SELECT adult.id AS adult_id, adult.age AS adult_age, adult.type_employer AS adult_type_employer, adult.fnlwgt AS adult_fnlwgt, adult.education AS adult_education, adult.sex AS adult_sex, adult.income AS adult_income 
FROM adult
2018-06-13 23:46:11,350 INFO sqlalchemy.engine.base.Engine ()


In [25]:
our_user = session.query(adult_data).filter_by(fnlwgt=71519).first() 

2018-06-13 23:46:46,262 INFO sqlalchemy.engine.base.Engine SELECT adult.id AS adult_id, adult.age AS adult_age, adult.type_employer AS adult_type_employer, adult.fnlwgt AS adult_fnlwgt, adult.education AS adult_education, adult.sex AS adult_sex, adult.income AS adult_income 
FROM adult 
WHERE adult.fnlwgt = ?
 LIMIT ? OFFSET ?
2018-06-13 23:46:46,269 INFO sqlalchemy.engine.base.Engine (71519, 1, 0)


In [39]:
our_user

<adult_data(age='39',fnlwgt='75165',education='Degree',sex='Female',income='>=50k')>

In [40]:
session.dirty

IdentitySet([])

In [41]:
session.new

IdentitySet([])

In [29]:
session.add_all([
...     adult_test(age=40, fnlwgt=74728, type_employer=' Private',education='Bachelors',sex='Male' , income='>=50k'),
...     adult_test(age=39, fnlwgt=75165, type_employer=' Private',education='Degree',sex='Female' , income='>=50k')])

In [30]:
our_user = session.query(adult_test).filter_by(age=39).first() 

2018-06-13 23:48:39,139 INFO sqlalchemy.engine.base.Engine INSERT INTO adult (age, type_employer, fnlwgt, education, sex, income) VALUES (?, ?, ?, ?, ?, ?)
2018-06-13 23:48:39,147 INFO sqlalchemy.engine.base.Engine (40, ' Private', 74728, 'Bachelors', 'Male', '>=50k')
2018-06-13 23:48:39,152 INFO sqlalchemy.engine.base.Engine INSERT INTO adult (age, type_employer, fnlwgt, education, sex, income) VALUES (?, ?, ?, ?, ?, ?)
2018-06-13 23:48:39,157 INFO sqlalchemy.engine.base.Engine (39, ' Private', 75165, 'Degree', 'Female', '>=50k')
2018-06-13 23:48:39,164 INFO sqlalchemy.engine.base.Engine SELECT adult.id AS adult_id, adult.age AS adult_age, adult.type_employer AS adult_type_employer, adult.fnlwgt AS adult_fnlwgt, adult.education AS adult_education, adult.sex AS adult_sex, adult.income AS adult_income 
FROM adult 
WHERE adult.age = ?
 LIMIT ? OFFSET ?
2018-06-13 23:48:39,169 INFO sqlalchemy.engine.base.Engine (39, 1, 0)


#### Filter data

In [32]:
for user in session.query(adult_test).filter(~adult_test.age.in_([39, 40])):
    print(user)

2018-06-13 23:49:04,641 INFO sqlalchemy.engine.base.Engine SELECT adult.id AS adult_id, adult.age AS adult_age, adult.type_employer AS adult_type_employer, adult.fnlwgt AS adult_fnlwgt, adult.education AS adult_education, adult.sex AS adult_sex, adult.income AS adult_income 
FROM adult 
WHERE adult.age NOT IN (?, ?)
2018-06-13 23:49:04,651 INFO sqlalchemy.engine.base.Engine (39, 40)
<adult_data(age='23',fnlwgt='77516',education=' Bachelors',sex=' Male',income=' >=50k')>
<adult_data(age='23',fnlwgt='71519',education=' Bachelors',sex=' Male',income=' >=50k')>


In [35]:
session.new

IdentitySet([])

In [34]:
session.commit()

2018-06-13 23:51:13,193 INFO sqlalchemy.engine.base.Engine COMMIT


In [37]:
our_user1 = session.query(adult_test).filter_by(age=39).first() 

2018-06-13 23:51:52,605 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-06-13 23:51:52,610 INFO sqlalchemy.engine.base.Engine SELECT adult.id AS adult_id, adult.age AS adult_age, adult.type_employer AS adult_type_employer, adult.fnlwgt AS adult_fnlwgt, adult.education AS adult_education, adult.sex AS adult_sex, adult.income AS adult_income 
FROM adult 
WHERE adult.age = ?
 LIMIT ? OFFSET ?
2018-06-13 23:51:52,612 INFO sqlalchemy.engine.base.Engine (39, 1, 0)


In [38]:
our_user1

<adult_data(age='39',fnlwgt='75165',education='Degree',sex='Female',income='>=50k')>

#### Delete Rows

In [42]:
q = session.query(adult_test).filter_by(age=39).one()
if q != []:
    session.delete(q)
    session.commit()

2018-06-13 23:53:17,051 INFO sqlalchemy.engine.base.Engine SELECT adult.id AS adult_id, adult.age AS adult_age, adult.type_employer AS adult_type_employer, adult.fnlwgt AS adult_fnlwgt, adult.education AS adult_education, adult.sex AS adult_sex, adult.income AS adult_income 
FROM adult 
WHERE adult.age = ?
2018-06-13 23:53:17,059 INFO sqlalchemy.engine.base.Engine (39,)
2018-06-13 23:53:17,066 INFO sqlalchemy.engine.base.Engine DELETE FROM adult WHERE adult.id = ?
2018-06-13 23:53:17,069 INFO sqlalchemy.engine.base.Engine (4,)
2018-06-13 23:53:17,079 INFO sqlalchemy.engine.base.Engine COMMIT


In [43]:
q = session.query(adult_test).filter_by(age=40).one()
if q != []:
    session.delete(q)
    session.commit()

2018-06-13 23:53:57,139 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-06-13 23:53:57,144 INFO sqlalchemy.engine.base.Engine SELECT adult.id AS adult_id, adult.age AS adult_age, adult.type_employer AS adult_type_employer, adult.fnlwgt AS adult_fnlwgt, adult.education AS adult_education, adult.sex AS adult_sex, adult.income AS adult_income 
FROM adult 
WHERE adult.age = ?
2018-06-13 23:53:57,147 INFO sqlalchemy.engine.base.Engine (40,)
2018-06-13 23:53:57,154 INFO sqlalchemy.engine.base.Engine DELETE FROM adult WHERE adult.id = ?
2018-06-13 23:53:57,159 INFO sqlalchemy.engine.base.Engine (3,)
2018-06-13 23:53:57,165 INFO sqlalchemy.engine.base.Engine COMMIT


In [44]:
for instance in session.query(adult_test).order_by(adult_test.id):
     print(instance.age, instance.fnlwgt)

2018-06-13 23:56:09,429 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-06-13 23:56:09,434 INFO sqlalchemy.engine.base.Engine SELECT adult.id AS adult_id, adult.age AS adult_age, adult.type_employer AS adult_type_employer, adult.fnlwgt AS adult_fnlwgt, adult.education AS adult_education, adult.sex AS adult_sex, adult.income AS adult_income 
FROM adult ORDER BY adult.id
2018-06-13 23:56:09,437 INFO sqlalchemy.engine.base.Engine ()
23 77516
23 71519


#### Add function

In [46]:
from sqlalchemy import func
session.query(func.count('*')).select_from(adult_test).scalar()

2018-06-13 23:58:11,939 INFO sqlalchemy.engine.base.Engine SELECT count(?) AS count_1 
FROM adult
2018-06-13 23:58:11,946 INFO sqlalchemy.engine.base.Engine ('*',)


2